**Run the demo.** Run the following cells to compute semantic correspondences for real image pairs.

In [1]:
import os

os.environ['http_proxy'] = "http://127.0.0.1:7891"
os.environ['https_proxy'] = "http://127.0.0.1:7891"

import requests

response = requests.get('http://www.google.com')
print(response.status_code)


200


In [ ]:
%load_ext autoreload
%autoreload 2
from archs.correspondence_utils import (
  process_image,
  rescale_points,
  draw_correspondences,
  compute_pck,
  find_nn_source_correspondences,
  find_best_buddies_correspondences
)
import os
import torch

import einops
import matplotlib.pyplot as plt

from PIL import Image
import numpy as np
import json
# import torchvision

from extract_hyperfeatures import load_models

In [ ]:
# Memory requirement is 13731MiB
device = "cuda"
config_path = "configs/real.yaml"
config, diffusion_extractor, aggregation_network = load_models(config_path, device)

In [ ]:
def load_image_pair(ann, load_size, device, image_path=""):
  img1_pil = Image.open(f"{image_path}/{ann['source_path']}").convert("RGB")
  img2_pil = Image.open(f"{image_path}/{ann['target_path']}").convert("RGB")
  source_size = img1_pil.size
  target_size = img2_pil.size

  # swap from (x, y) to (y, x)
  if "source_points" in ann:
    source_points, target_points = ann["source_points"], ann["target_points"]
    source_points = np.flip(source_points, 1)
    target_points = np.flip(target_points, 1)
    source_points = rescale_points(source_points, source_size, load_size)
    target_points = rescale_points(target_points, target_size, load_size)
  else:
    source_points, target_points = None, None

  img1, img1_pil = process_image(img1_pil, res=load_size)
  img2, img2_pil = process_image(img2_pil, res=load_size)
  img1, img2 = img1.to(device), img2.to(device)
  imgs = torch.cat([img1, img2])

  return source_points, target_points, img1_pil, img2_pil, imgs

def load_saliency_pair(ann, output_size, device, saliency_path=""):
  def _load_saliency(path):
    if os.path.exists(path):
      saliency_pil = Image.open(path).convert("L")
      saliency_map, _ = process_image(saliency_pil, res=output_size, range=(0, 1))
      saliency_map = einops.rearrange(saliency_map, 'b c h w -> (b c) (h w)')
    else:
      saliency_map = torch.ones((1, output_size[0] * output_size[1]))
    saliency_map = saliency_map.to(device)
    return saliency_map
  saliency_map1 = _load_saliency(f"{saliency_path}/{ann['source_path']}")
  saliency_map2 = _load_saliency(f"{saliency_path}/{ann['target_path']}")
  return saliency_map1, saliency_map2

def reshape_descriptors(img1_feats, img2_feats):
  b, d, w, h = img1_feats.shape
  descriptors1 = img1_feats.view((b, d, -1)).permute((0, 2, 1))[:, None, ...]
  descriptors2 = img2_feats.view((b, d, -1)).permute((0, 2, 1))[:, None, ...]
  return descriptors1, descriptors2

def postprocess_points(source_points, predicted_points, output_size, load_size):
  source_points = source_points.detach().cpu().numpy()
  predicted_points = predicted_points.detach().cpu().numpy()
  source_points = rescale_points(source_points, output_size, load_size)
  predicted_points = rescale_points(predicted_points, output_size, load_size)
  return source_points, predicted_points

**Compute semantic keypoint matches.** Use one of the correspondence functions:
- `nearest_neighbors`: For a set of annotated source_points, compute the nearest neighbor predicted_points.
- `best_buddies`: Compute the top-k pairs of source_points, predicted_points using the [best buddies algorithm](https://github.com/ShirAmir/dino-vit-features) (Amir et. al., ECCVW 2022).

In [ ]:
image_path = "assets/spair/images"
saliency_path = "assets/spair/odise"
annotation_path = "annotations/spair_71k_test-6.json"
load_size = (224, 224)
output_size = (config["output_resolution"], config["output_resolution"])

# Select from [nearest_neighbors, best_buddies]
correspondence_function = "nearest_neighbors"
# Number of correspondences to display when using [best_buddies]
num_pairs = 10

In [ ]:
for ann in json.load(open(annotation_path)):
  source_points, target_points, img1_pil, img2_pil, imgs = load_image_pair(ann, load_size, device, image_path)
  
  with torch.inference_mode():
    with torch.autocast("cuda"):
      feats, _ = diffusion_extractor.forward(imgs)
      b, s, l, w, h = feats.shape
      diffusion_hyperfeats = aggregation_network(feats.float().view((b, -1, w, h)))
      img1_hyperfeats = diffusion_hyperfeats[0][None, ...]
      img2_hyperfeats = diffusion_hyperfeats[1][None, ...]

  if correspondence_function == "nearest_neighbors":
    _, predicted_points = find_nn_source_correspondences(img1_hyperfeats, img2_hyperfeats, source_points, output_size, load_size)
    predicted_points = predicted_points.detach().cpu().numpy()
    distances, _, pck_metric = compute_pck(predicted_points, target_points, load_size)
    title = f"Diffusion Hyperfeatures, Nearest Neighbors Matches \n PCK@0.1: {pck_metric.round(decimals=2)}"
  elif correspondence_function == "best_buddies":
    descriptors1, descriptors2 = reshape_descriptors(img1_hyperfeats, img2_hyperfeats)
    saliency_map1, saliency_map2 = load_saliency_pair(ann, output_size, img1_hyperfeats.device, saliency_path)
    source_points, predicted_points = find_best_buddies_correspondences(descriptors1, descriptors2, saliency_map1, saliency_map2, num_pairs=num_pairs)
    source_points, predicted_points = postprocess_points(source_points, predicted_points, output_size, load_size)
    title = "Diffusion Hyperfeatures, Best Buddies Matches"
  else:
    raise NotImplementedError
  
  draw_correspondences(source_points, predicted_points, img1_pil, img2_pil, title=title)
  plt.show()